In [1]:
import pandas as pd
df = pd.read_csv('food_reviews_sample.csv')

In [3]:
corpus = list(df['text'].values)
# list of strings

### Naive Approach For Full Text Search
- Serial Search
- Analogy: Flipping through a dictionary from beginning to end to find a word.

In [5]:
import re

In [7]:
def simple_search(word, corpus:list):
    
    list_of_docs = []    
    for i, doc in enumerate(corpus):      
        for w in re.findall(f'[a-z]+', doc.lower()):        
            if w == word:              
                list_of_docs.append(i)
    return list_of_docs

In [12]:
%time result = simple_search('chocolate', corpus)

CPU times: user 299 ms, sys: 6.45 ms, total: 306 ms
Wall time: 319 ms


### Smarter Way: Using an Index
- Inverted Index
- {'word' : [doc1, doc2, doc3]}

In [21]:
def create_index(corpus):
    
    index = {}
    for i, doc in enumerate(corpus):
        for w in re.findall(f'[a-z]+', doc.lower()):
            if w in index: #if we've already seen the word before
                index[w].append(i)
            else:
                index[w] = [i] 
    return index

In [24]:
%time index = create_index(corpus)

CPU times: user 495 ms, sys: 15.3 ms, total: 510 ms
Wall time: 523 ms


In [26]:
%timeit result = index['chocolate']

73.8 ns ± 2.4 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


- Naive Search: ~300ms
- Indexed Search: ~70 ns

Limitations:
- Build the index frequently. Maybe not so good for dynamic data.
- No concept of natural language (e.g. lemmatization, stop words)
- We cant do phrase search / multiple words. 
- Typos (Fuzzy searching)
- Ranking?!

---
## Full Text Search in PostGres
---

FTS representation in PostGres:
- We have a datatype called `tsvector`, which basically is pre-processed and stripped down to a basic form (e.g. lemmatized)

- We then have `tsquery`, which is a search query that is normalized into lexemes / lemmas.

- In order to match a query to a vector, we have to use the strange `@@` syntax.

### Example Queries, Part 1 (on individual strings first, not the data set yet):

**(1) Basic Matching**: `SELECT to_tsvector('this dish was horrible') @@ to_tsquery('horrible');`
- What's nice is that it's case insensitive! So it automatically does lowercase.

**(2) Multiple words**: `SELECT to_tsvector('this dish was horrible') @@ plainto_tsquery('horrible dish');`
- *plainto_tsquery* transforms unformatted text querytext to tsquery. The text is parsed and normalized much as for to_tsvector, then the & (AND) Boolean operator is inserted between surviving words.

**(3) Under the Hood**: `SELECT to_tsvector('this dish was horrible');`
- Note that it returns the tokenized sentence! Even does lemmatiziation for us (e.g. horrible -> horribl)

**(4) Interesting case**: `SELECT to_tsvector('A nice day for a car ride') @@ plainto_tsquery('I am riding a bike'); `
- Returns False! Interesting, because it contains extra information in bike and therefore does not match. Bike isn’t included therefore the document isn’t relevant. But **I am riding** DOES work!

---

### Example Queries, Part 2 (on the food review data set!):

**(5) Searching a column**: `SELECT id, text FROM food_reviews WHERE to_tsvector(text) @@ to_tsquery('horrible');`
- Works, but it's slow (took ~2.5 seconds to return all results)

**(6) Another**: `SELECT id,text FROM food_reviews 
WHERE to_tsvector(text) @@ plainto_tsquery('dry pasta');`

**(7) Concatenating Multiple Cols**: `SELECT summary,text FROM food_reviews 
WHERE to_tsvector(summary || ' ' || text) @@ plainto_tsquery('addictive flavor');`

**(8) Enforce Word Order**: `SELECT text FROM food_reviews
WHERE to_tsvector(text) @@ to_tsquery('addictive <-> flavor');`
- Only return results where "addictive" is immediately followed by "flavor"

---

### Example Queries, Part 3 (on the food review data set!):
- **Indexing and Ranking!**

**(9) Create an Inverted Index**:

- The coalesce function will default to `' '` (empty string) if the text contains NA values.
- The creation of the index took about 6 seconds!!

**(10) Query on the Index**:

- I'm sure there's a better way to give that whole index a nickname but I didn't have time to figure it out.
- The query took about **0.7 milliseconds!!**
    - If you run `EXPLAIN ANALYZE` before the query.

**(11) Ranking the results**:
- Disclaimer: I don't exactly know the mechanics of the algorithm behind how it's ranking the results, but from skimming the PostGres documention, it looks like it's fairly well-explained.

---

### Other cool things that I found in PostGres FTS (haven't run the code myself though):
- `pg_trgm` (trigram indexing operations)

- unaccent (removes accents, umlauts, etc.)

- Fuzzy String matching
    - e.g. `SELECT name FROM users WHERE levenshtein(‘Stephen’, name) <=2;`
    
    - Uses levenshtein distances to return words that are similar / maybe had typos.
    
- PostGres support JSON objects, and there's even aquerying language for JSON (JsQuery) built directly into PostGres.